# PowerWorld Problem 2 - Maximum Design Faults 
# and Circuit Breaker Design
Author: David Stump, PE

This Jupyter notebook, written in Python, is a supplement to the PowerWorld project problem 2 as required as part of EE 443 at the University of Southern California in the fall of 2017. The problem is based off of Example 7.6 from Glover's 6th Edition of Power System Analysis and Design. 
<img src="original circuit.png",width=300>


In [2]:
import numpy as np
import pandas as pd 

### Definition of Givens


In [3]:
X_TL54 = 0.00025j # PU transmission line impedance from bus 5 to bus 4
X_TL52 = 0.0005j # PU transmission line impedance from bus 5 to bus 2
X_TL42B = 0.001j # PU transmission line impedance from bus 4 to bus 2, circuit breaker B
X_TL42C = 0.00075j # PU transmission line impedance from bus 4 to bus 2, circuit breaker C
X_T1 = 0.0002j # PU internal impedance of Transformer 1
X_T2 = 0.0001j # PU internal impedance of Transformer 1
X_G1 = 0.045j # PU internal impedance of Generator 1
X_G2 = 0.0225j # PU internal impedance of Generator 2

### Delta to Wye Conversion
Both of the superposition cases will require the delta structure $X_{TL52}$, $X_{TL54}$, and $X_{TL42B}$ to be transformed to a wye so that we can simplify the structure and fine $X_{TH}$.


In [4]:
Z_sum = X_TL52+X_TL54+X_TL42B
Z1 = X_TL52*X_TL54/Z_sum
Z2 = X_TL52*X_TL42B/Z_sum
Z3 = X_TL54*X_TL42B/Z_sum

In [5]:
Z1

7.142857142857142e-05j

In [6]:
Z2

0.0002857142857142857j

In [7]:
Z3

0.00014285714285714284j

### PU Analysis Table
This per unit analysis table is required for both superposition cases as well. 

In [8]:
S = 1*10**6 #VA
V_B1 = 15000. #volts
V_B2 = 15000. #volts
V_B3 = 345000. #volts

In [9]:
pu_df2 = pd.DataFrame({ 'Zone' : ['One','Two','Three'],'S_Base' : [S,S,S], 'V_Base' : [V_B1, V_B2, V_B3]})
pu_df2['I_Base'] = pu_df2['S_Base']/(np.sqrt(3)*pu_df2['V_Base']) # the base current per zone
pu_df2['Z_Base'] = pu_df2['V_Base']**2/pu_df2['S_Base'] # the base resistance per zone
pu_df2['V_Op'] = pu_df2['V_Base']*1.05 # the operating voltage per zone. Note it is 5% above V_Base
pu_df2 # the per zone dataframe for the PU analysis


,S_Base,V_Base,Zone,I_Base,Z_Base,V_Op
0,1000000,15000.0,One,38.490018,225.0,15750.0
1,1000000,15000.0,Two,38.490018,225.0,15750.0
2,1000000,345000.0,Three,1.673479,119025.0,362250.0


## Simplification of the Network

### Superposition Case 1
In the first case we will disable the first generator and solve for the total short circuit current. We will still have to solve the original circuit with the delta configuration to find the current values in the tranmission lines. 
<img src="superposition case 1.png",width=400>


In [ ]:
a1 = X_G1+X_T1+Z1
d1 = 1/(1/X_TL42B+1/X_TL42C+1/(X_T2+X_G2))
b1 = Z3 + d1                
c1 = 1/(1/Z2+1/b1) 

In [11]:
X_eq1 = a1+c1
X_eq1 #pu

0.04546101022750991j

In [12]:
pu_df2['I_sc_pu_1'] = 1.05/(X_eq1)
pu_df2['I_sc_1'] = pu_df2['I_sc_pu_1']*pu_df2['I_Base']
pu_df2

,S_Base,V_Base,Zone,I_Base,Z_Base,V_Op,I_sc_pu_1,I_sc_1
0,1000000,15000.0,One,38.490018,225.0,15750.0,-23.0967150696j,-888.992977521j
1,1000000,15000.0,Two,38.490018,225.0,15750.0,-23.0967150696j,-888.992977521j
2,1000000,345000.0,Three,1.673479,119025.0,362250.0,-23.0967150696j,-38.6518685879j


In [13]:
I_sc_pu_1 = pu_df2.get_value(2,'I_sc_pu_1')
I_sc_pu_1 #pu

-23.09671506957871j

### Current Divider: Superposition Case 1
Now we have to resolve the first case with the original structure to find the current in the transmission lines. 
<img src="current divider case 1.png",width=400>


In [33]:
def parallel(*args):
    """
    This is a function that returns the value of an unknown number of parallel
    impedances. 
    """
    Z=[None]*len(args)
    for i in np.arange(len(args)):
        Z[i]=1/args[i]
    return(1/sum(Z))

In [35]:
X_isc_A1_pu = parallel(X_TL52,(X_TL54+parallel(X_G2+X_T2, X_TL42B, X_TL42C)))
X_isc_A1_pu

(-0+0.00028643349231584526j)

In [36]:
I_sc_A1_pu = I_sc_pu_1*X_isc_A1_pu/X_TL52
I_sc_A1_pu

-13.231345516806883j

In [44]:
I_sc_B1_pu = ((I_sc_pu_1-I_sc_A1_pu)*parallel(X_G2+X_T2, X_TL42B, X_TL42C)/
              X_TL42C)
I_sc_B1_pu #pu

-5.5324404936139775j

In [45]:
I_sc_C1_pu = ((I_sc_pu_1-I_sc_A1_pu)*parallel(X_G2+X_T2, X_TL42B, X_TL42C)/
              X_TL42B)
I_sc_C1_pu #pu

-4.1493303702104836j

## Superposition Case 2
In the second superposition case, I have disabled the first generator and converted the delta to a wye. 
<img src="superposition case 2.png",width=400>

### PU Analysis Table

In [18]:
a2 = X_G2+X_T2
b2 = Z3+1/(1/Z2+1/a1)
c2 = 1/(1/X_TL42B+1/X_TL42C+1/b2)
d2 = 1/(1/a1+1/Z2)
X_eq2 = a2 + c2
X_eq2 #pu

0.022813836807374043j

In [20]:
pu_df2['I_sc_pu_2'] = 1.05/(X_eq2)
pu_df2['I_sc_2'] = pu_df2['I_sc_pu_2']*pu_df2['I_Base']
pu_df2

,S_Base,V_Base,Zone,I_Base,Z_Base,V_Op,I_sc_pu_1,I_sc_1,I_sc_pu_2,I_sc_2
0,1000000,15000.0,One,38.490018,225.0,15750.0,-23.0967150696j,-888.992977521j,-46.0247002232j,-1771.49153755j
1,1000000,15000.0,Two,38.490018,225.0,15750.0,-23.0967150696j,-888.992977521j,-46.0247002232j,-1771.49153755j
2,1000000,345000.0,Three,1.673479,119025.0,362250.0,-23.0967150696j,-38.6518685879j,-46.0247002232j,-77.0213711978j


In [21]:
I_sc_pu_2 = pu_df2.get_value(2,'I_sc_pu_2')
I_sc_pu_2 #pu

-46.02470022318262j

### Current Divider: Superposition Case 2
Now that we have the total current through the circuit, we can use the current divider method to solve for the currents in the transmission lines. 
<img src="current divider case 2.png",width=400>

In [43]:
I_sc_C2_pu = (I_sc_pu_2*parallel(X_TL42C, X_TL42B, X_TL54+
                                 parallel(X_G1+X_T1, X_TL52))/X_TL42C)
I_sc_C2_pu #pu

-16.691657318966502j

In [46]:
I_sc_B2_pu = (I_sc_pu_2*parallel(X_TL42C, X_TL42B, X_TL54+
                                 parallel(X_G1+X_T1, X_TL52))/X_TL42B)
I_sc_B2_pu #pu

-12.518742989224878j

In [47]:

I_sc_A2_pu = ((I_sc_pu_2-I_sc_B2_pu - I_sc_C2_pu)*
              parallel(X_G1+X_T1, X_TL52)/X_TL52)
I_sc_A2_pu #pu

-16.63033602095414j

### Composition of the two cases

In [48]:
I_sc_A = I_sc_A1_pu + I_sc_A2_pu
I_sc_A

-29.861681537761022j

In [49]:
I_sc_B = I_sc_B1_pu + I_sc_B2_pu
I_sc_B

-18.051183482838855j

In [50]:
I_sc_C = I_sc_C1_pu + I_sc_C2_pu
I_sc_C

-20.840987689176984j

In [51]:
I_sc = I_sc_A+I_sc_B+I_sc_C
I_sc

-68.753852709776865j

### Calculation of Actual Currents

In [53]:
I_sc_A_actual = I_sc_A*pu_df2.get_value(2, 'I_Base')
I_sc_A_actual

-49.972898186322332j

In [54]:
I_sc_B_actual = I_sc_B*pu_df2.get_value(2, 'I_Base')
I_sc_B_actual

-30.208277226111129j

In [55]:
I_sc_C_actual = I_sc_C*pu_df2.get_value(2, 'I_Base')
I_sc_C_actual

-34.876956094272487j